In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
path_data = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/freeze_191003/TBIFreeze_20191003_python.xlsx'

#original file has multiple sheets (for different data set types (e.g. single visit, imaging etc)
#only interested in deploayed controls ('C') and mTBI ('T')
#each participant can have multiple visits - we are currently intersted in data only from the first visit

#we want to create one data frame with one row per participant corresponding to their first study visit and data across columns
first_visit_data = pd.DataFrame()

whole_file = pd.ExcelFile(path_data)
print(whole_file.sheet_names, '\n')

#each data set uses as different column name for visit date, use a dictionary to access corresponding date column name
sheet_dic = {'Single_Record_Measures': 'ScreenDate', 
             'Multi_Record_Measures': 'MeasureDate', 
             'TBI_Symptom_Q': 'NSIFormDate',  
             'Test_Dates': 'CogPreTestDate',
             'Clinical_Labs': 'CLabDate',
             'DTI_Kleinhans_Lab': 'ScreenDate', 
             'FDG-PET': 'ScreenDate'}

for sheet in whole_file.sheet_names:
    print('Sheet being processed:\n', sheet)
    
    #create intermediate dataframe
    data_int = pd.DataFrame(data = pd.read_excel(whole_file, sheet))

    #select only TBIID C and T (control and TBI)
    data_int = data_int[data_int['TBIID'].str.match(r'[CT]\d\d')]
    print('Data shape only deployed controls and mTBI groups:\n', data_int.shape)

    # create new data frame containing only the first visit record (based on 'ScreenDate' column)
    #add new column with group ID (C = deployed controls, T = blast mTBI)
    first_visit_data_int = pd.DataFrame()

    participants = data_int['TBIID'].unique()
    print('Number of participants:\n', len(participants))

    for part in participants:
        dates = data_int.loc[data_int['TBIID'] == part, sheet_dic[sheet]].values
        min_date = dates.min()
    
        first_date = data_int[(data_int['TBIID'] == part) & (data_int[sheet_dic[sheet]] == min_date)]
        first_visit_data_int = first_visit_data_int.append(first_date)
                

    #reset indexes and add to final dataframe
    first_visit_data_int = first_visit_data_int.reset_index(drop=True)
    #clean up missing values
    first_visit_data_int = first_visit_data_int.replace({-999.0: np.nan, 999: np.nan, 'None': np.nan})
    
    if first_visit_data.shape[0] < 1:
        first_visit_data = first_visit_data_int
    else:
        first_visit_data = pd.merge(first_visit_data, first_visit_data_int, how='left', on=['TBIID'], sort=False)
    print('First visit data shape for current sheet:\n', first_visit_data.shape, '\n')

#clean up duplicates
first_visit_data.drop_duplicates(inplace=True)

#add group column 
first_visit_data['Group'] = [TBIID[0] for TBIID in first_visit_data['TBIID']]

print('Final shape of first visit data:\n', first_visit_data.shape)
first_visit_data.head()

['Single_Record_Measures', 'Multi_Record_Measures', 'TBI_Symptom_Q', 'Test_Dates', 'Clinical_Labs', 'DTI_Kleinhans_Lab', 'FDG-PET'] 

Sheet being processed:
 Single_Record_Measures
Data shape only deployed controls and mTBI groups:
 (215, 835)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 835) 

Sheet being processed:
 Multi_Record_Measures
Data shape only deployed controls and mTBI groups:
 (271, 84)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 918) 

Sheet being processed:
 TBI_Symptom_Q
Data shape only deployed controls and mTBI groups:
 (265, 42)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 959) 

Sheet being processed:
 Test_Dates
Data shape only deployed controls and mTBI groups:
 (257, 61)
Number of participants:
 144
First visit data shape for current sheet:
 (145, 1019) 

Sheet being processed:
 Clinical_Labs
Data shape only deployed controls and mTBI groups:
 (318, 21)
Number of 

,EntityID,DOB,GType,Race,Hispanic,Handedness,Status_x,StatusChangedDate,KFPSubject,TBIID,ScreenDate_x,VisitSeq_x,ScreenAge,Education,branchAF,branchArm,branchCG,branchMar,branchNav,branchNG,branchOth,MOS,MOS2,MOS3,MOS4,MOS5,Marital,servconn,cestotal,ExpPB,ExpMark1,ExpAntiM,ExpStim,ExpOthrS,ExpDEET,ExpTick,ExpPCollr,ExpPStrp,ExpPEnvi,ExpToxic,ExpPaint,ExpXsVib,ExpHStrk,ExpRadar,ExpIonRa,ExpYCake,ExpVhicl,ExpUrRnd,ExpDtOrd,ExpGasM,ExpMOPP,ExpRadBg,ExpAN,ExpNG,ExpTNT,ExpPETN,ExpRDX,ExpNC,ExpANFO,ExpCompB,ExpOctol,ExpPntlt,ExpDynmt,ExpOthrX,TBIMilitaryTBIID,StartDate1,EndDate1,StartDate2,EndDate2,StartDate3,EndDate3,StartDate4,EndDate4,StartDate5,EndDate5,CognitiveTBIID,PsyEduc,MMSEORDA,MMSEORLO,MMSE,DIGIF3,DIGIFLEN3,DIGIB3,DIGIBLEN3,DIGITOT3,DIGIPER3,ANIMALS,TRAILA,TRAILB,LETTERS,LETTERF,LETTERA,AnimalsT,TrailAT,TrailBT,LetFAST,wtarfsiq,wtarviq,tommtot1,tommtot2,tommret,CVTr1FR,CVTr1FRT,CVT15FR,CVT15FRT,CVShDFR,CVShDFRT,CVShDCR,CVShDCRT,CVLoDFR,CVLoDFRT,CVLoDCR,CVLoDCRT,CVYNRH,CVYNRHT,CVYNFP,CVYNFPT,CVFCRRaw,CVDeInt,BVTr1,BVTr1T,BVTotRe,BVTotReT,BVLearn,BVLearnT,BVDelRe,BVDelReT,BVRecHit,BVRecFA,BVDeInt,FTDomH,FTDomHT,FTNDomH,FTNDomHT,W3LNS,W3LNSSS,GPgD,GPgDT,GPgND,GPgNDT,CPTOm,CPTOmT,CPTCom,CPTComT,CPTHRT,CPTHRTT,CPTHSE,CPTHSET,CPTVar,CPTVarT,CPTd,CPTdT,CPTB,CPTBT,CPTRTBl,CPTRTBlT,CPTSEBl,CPTSEBlT,CPTRTIs,CPTRTIsT,CPTSEIs,CPTSEIsT,WCSTPrs,WCSTPrsT,WCSTCL,WCSTCLP,WCSTCLPT,WCSTCat,WCSTFail,TowerItemAd,TowerFrstMove,TowerMove,NumItemAdmin,TowerRuleVio,TowerItemCom,TowerAchieve,TowerAchieveS,TowerMeanFrstMoveR,TowerMeanFrstMoveRS,TowerTimePerMoveR,TowerTimePerMoveRS,TowerMoveAccR,TowerMoveAccRS,TowerRuleVioPerc,TEA6tptS,TEA7dtdS,TEA8LotS,SDW90Raw,SDW90TS,SDO90Raw,SDO90TS,MTRRRaw,MTRRRPer,MT2Raw,MT2Per,MT15Raw,MT15Per,MTTCRaw,MTTCPer,MTECRaw,MTECPer,MTVRRaw,MTVRPer,MTARRaw,MTARPer,MTPMTRaw,MTPMTPer,MTDPRaw,MTClck,MTPMT1EC,MTPMT2EC,MTPMT3EC,MTPMT4EC,MTPMT5EC,MTPMT6EC,MTPMT7EC,MTPMT8EC,CvmtHitR,CvmtHitT,CvmtFaR,CvmtFaT,CvmtDprR,CvmtDprT,CvmtTotR,CvmtTotT,CvmtDelR,CvmtDelT,CvmtVis,CvmtInt,DkefCnR,DkefCnSS,DkefWrR,DkefWrSS,DkefInR,DkefInSS,DkefSwR,DkefSwSS,DkefCnEr,DkefWrEr,DkefInEr,DkefSwEr,DkefCnUn,DkefWrUn,DkefInUn,DkefSwUn,NDCompSc,NDRdRate,ACTFrmDate,ACT0,ACT9,...,DTI_MD_retrolenticular_int_capsule_L,DTI_MD_ant_corona_radiata_R,DTI_MD_ant_corona_radiata_L,DTI_MD_sup_corona_radiata_R,DTI_MD_sup_corona_radiata_L,DTI_MD_post_corona_radiata_R,DTI_MD_post_corona_radiata_L,DTI_MD_post_thal_rad_R,DTI_MD_post_thal_rad_L,DTI_MD_sag_stratum_R,DTI_MD_sag_stratum_L,DTI_MD_ext_capsule_R,DTI_MD_ext_capsule_L,DTI_MD_cingulum_cg_R,DTI_MD_cingulum_cg_L,DTI_MD_cingulum_hipp_R,DTI_MD_cingulum_hipp_L,DTI_MD_fornix_striaterm_R,DTI_MD_fornix_striaterm_L,DTI_MD_sup_long_fascic_R,DTI_MD_sup_long_fascic_L,DTI_MD_sup_frontocc_fascic_R,DTI_MD_sup_frontocc_fascic_L,DTI_MD_unc_fascic_R,DTI_MD_unc_fascic_L,DTI_MD_tapetum_R,DTI_MD_tapetum_L,subjects_,DTI_AD_mid_cerebll_pedunc,DTI_AD_pontine_xing_tract,DTI_AD_genu_cc,DTI_AD_body_cc,DTI_AD_splenium_cc,DTI_AD_fornix,DTI_AD_corticospinal_R,DTI_AD_corticospinal_L,DTI_AD_med_lemniscus_R,DTI_AD_med_lemniscus_L,DTI_AD_inf_cerebll_pedunc_R,DTI_AD_inf_cerebll_pedunc_L,DTI_AD_sup_cerebll_pedunc_R,DTI_AD_sup_cerebll_pedunc_L,DTI_AD_cerebr_pedunc_R,DTI_AD_cerebr_pedunc_L,DTI_AD_ant_int_capsule_R,DTI_AD_ant_int_capsule_L,DTI_AD_post_int_capsule_R,DTI_AD_post_int_capsule_L,DTI_AD_retrolenticular_int_capsule_R,DTI_AD_retrolenticular_int_capsule_L,DTI_AD_ant_corona_radiata_R,DTI_AD_ant_corona_radiata_L,DTI_AD_sup_corona_radiata_R,DTI_AD_sup_corona_radiata_L,DTI_AD_post_corona_radiata_R,DTI_AD_post_corona_radiata_L,DTI_AD_post_thal_rad_R,DTI_AD_post_thal_rad_L,DTI_AD_sag_stratum_R,DTI_AD_sag_stratum_L,DTI_AD_ext_capsule_R,DTI_AD_ext_capsule_L,DTI_AD_cingulum_cg_R,DTI_AD_cingulum_cg_L,DTI_AD_cingulum_hipp_R,DTI_AD_cingulum_hipp_L,DTI_AD_fornix_striaterm_R,DTI_AD_fornix_striaterm_L,DTI_AD_sup_long_fascic_R,DTI_AD_sup_long_fascic_L,DTI_AD_sup_frontocc_fascic_R,DTI_AD_sup_frontocc_fascic_L,DTI_AD_unc_fascic_R,DTI_AD_unc_fascic_L,DTI_A

In [4]:
#read in data from RH pull and combine with df
path_RH = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/PeskindTBI/RHdatapull/RHpull.csv'

data_RH = pd.read_csv(path_RH)
data_RH = pd.DataFrame(data = data_RH)
print('Data shape all groups:\n', data_RH.shape)

#select only TBIID C and T (control and TBI)
data_RH = data_RH[data_RH['TBIID'].str.match(r'[CT]\d\d')]
print('Data shape only deployed controls and mTBI groups:\n', data_RH.shape)

#clean up missing values
data_RH = data_RH.replace({-999.0: np.nan, 'None': np.nan})

#change DA = 0 to np.nan, add ratio cals
data_RH['DA'] = data_RH['DA'].replace({0: np.nan})
data_RH['da_dopa_ratio'] = data_RH['DA'] / data_RH['DOPA']
data_RH['dopac_da_ratio'] = data_RH['DOPAC'] / data_RH['DA']
data_RH['ne_dopa_ratio'] = data_RH['NE'] / data_RH['DOPA']
data_RH['dhpg_ne_ratio'] = data_RH['DHPG'] / data_RH['NE']

#add to df containing first visit info
first_visit_data = pd.merge(first_visit_data, data_RH, how='left', on=['TBIID'], sort=False)

print('Final shape of first visit data:\n', first_visit_data.shape)
first_visit_data.head()

Data shape all groups:
 (194, 14)
Data shape only deployed controls and mTBI groups:
 (166, 14)
Final shape of first visit data:
 (144, 1377)


,EntityID,DOB,GType,Race,Hispanic,Handedness,Status_x,StatusChangedDate,KFPSubject,TBIID,ScreenDate_x,VisitSeq_x,ScreenAge,Education,branchAF,branchArm,branchCG,branchMar,branchNav,branchNG,branchOth,MOS,MOS2,MOS3,MOS4,MOS5,Marital,servconn,cestotal,ExpPB,ExpMark1,ExpAntiM,ExpStim,ExpOthrS,ExpDEET,ExpTick,ExpPCollr,ExpPStrp,ExpPEnvi,ExpToxic,ExpPaint,ExpXsVib,ExpHStrk,ExpRadar,ExpIonRa,ExpYCake,ExpVhicl,ExpUrRnd,ExpDtOrd,ExpGasM,ExpMOPP,ExpRadBg,ExpAN,ExpNG,ExpTNT,ExpPETN,ExpRDX,ExpNC,ExpANFO,ExpCompB,ExpOctol,ExpPntlt,ExpDynmt,ExpOthrX,TBIMilitaryTBIID,StartDate1,EndDate1,StartDate2,EndDate2,StartDate3,EndDate3,StartDate4,EndDate4,StartDate5,EndDate5,CognitiveTBIID,PsyEduc,MMSEORDA,MMSEORLO,MMSE,DIGIF3,DIGIFLEN3,DIGIB3,DIGIBLEN3,DIGITOT3,DIGIPER3,ANIMALS,TRAILA,TRAILB,LETTERS,LETTERF,LETTERA,AnimalsT,TrailAT,TrailBT,LetFAST,wtarfsiq,wtarviq,tommtot1,tommtot2,tommret,CVTr1FR,CVTr1FRT,CVT15FR,CVT15FRT,CVShDFR,CVShDFRT,CVShDCR,CVShDCRT,CVLoDFR,CVLoDFRT,CVLoDCR,CVLoDCRT,CVYNRH,CVYNRHT,CVYNFP,CVYNFPT,CVFCRRaw,CVDeInt,BVTr1,BVTr1T,BVTotRe,BVTotReT,BVLearn,BVLearnT,BVDelRe,BVDelReT,BVRecHit,BVRecFA,BVDeInt,FTDomH,FTDomHT,FTNDomH,FTNDomHT,W3LNS,W3LNSSS,GPgD,GPgDT,GPgND,GPgNDT,CPTOm,CPTOmT,CPTCom,CPTComT,CPTHRT,CPTHRTT,CPTHSE,CPTHSET,CPTVar,CPTVarT,CPTd,CPTdT,CPTB,CPTBT,CPTRTBl,CPTRTBlT,CPTSEBl,CPTSEBlT,CPTRTIs,CPTRTIsT,CPTSEIs,CPTSEIsT,WCSTPrs,WCSTPrsT,WCSTCL,WCSTCLP,WCSTCLPT,WCSTCat,WCSTFail,TowerItemAd,TowerFrstMove,TowerMove,NumItemAdmin,TowerRuleVio,TowerItemCom,TowerAchieve,TowerAchieveS,TowerMeanFrstMoveR,TowerMeanFrstMoveRS,TowerTimePerMoveR,TowerTimePerMoveRS,TowerMoveAccR,TowerMoveAccRS,TowerRuleVioPerc,TEA6tptS,TEA7dtdS,TEA8LotS,SDW90Raw,SDW90TS,SDO90Raw,SDO90TS,MTRRRaw,MTRRRPer,MT2Raw,MT2Per,MT15Raw,MT15Per,MTTCRaw,MTTCPer,MTECRaw,MTECPer,MTVRRaw,MTVRPer,MTARRaw,MTARPer,MTPMTRaw,MTPMTPer,MTDPRaw,MTClck,MTPMT1EC,MTPMT2EC,MTPMT3EC,MTPMT4EC,MTPMT5EC,MTPMT6EC,MTPMT7EC,MTPMT8EC,CvmtHitR,CvmtHitT,CvmtFaR,CvmtFaT,CvmtDprR,CvmtDprT,CvmtTotR,CvmtTotT,CvmtDelR,CvmtDelT,CvmtVis,CvmtInt,DkefCnR,DkefCnSS,DkefWrR,DkefWrSS,DkefInR,DkefInSS,DkefSwR,DkefSwSS,DkefCnEr,DkefWrEr,DkefInEr,DkefSwEr,DkefCnUn,DkefWrUn,DkefInUn,DkefSwUn,NDCompSc,NDRdRate,ACTFrmDate,ACT0,ACT9,...,DTI_MD_fornix_striaterm_R,DTI_MD_fornix_striaterm_L,DTI_MD_sup_long_fascic_R,DTI_MD_sup_long_fascic_L,DTI_MD_sup_frontocc_fascic_R,DTI_MD_sup_frontocc_fascic_L,DTI_MD_unc_fascic_R,DTI_MD_unc_fascic_L,DTI_MD_tapetum_R,DTI_MD_tapetum_L,subjects_,DTI_AD_mid_cerebll_pedunc,DTI_AD_pontine_xing_tract,DTI_AD_genu_cc,DTI_AD_body_cc,DTI_AD_splenium_cc,DTI_AD_fornix,DTI_AD_corticospinal_R,DTI_AD_corticospinal_L,DTI_AD_med_lemniscus_R,DTI_AD_med_lemniscus_L,DTI_AD_inf_cerebll_pedunc_R,DTI_AD_inf_cerebll_pedunc_L,DTI_AD_sup_cerebll_pedunc_R,DTI_AD_sup_cerebll_pedunc_L,DTI_AD_cerebr_pedunc_R,DTI_AD_cerebr_pedunc_L,DTI_AD_ant_int_capsule_R,DTI_AD_ant_int_capsule_L,DTI_AD_post_int_capsule_R,DTI_AD_post_int_capsule_L,DTI_AD_retrolenticular_int_capsule_R,DTI_AD_retrolenticular_int_capsule_L,DTI_AD_ant_corona_radiata_R,DTI_AD_ant_corona_radiata_L,DTI_AD_sup_corona_radiata_R,DTI_AD_sup_corona_radiata_L,DTI_AD_post_corona_radiata_R,DTI_AD_post_corona_radiata_L,DTI_AD_post_thal_rad_R,DTI_AD_post_thal_rad_L,DTI_AD_sag_stratum_R,DTI_AD_sag_stratum_L,DTI_AD_ext_capsule_R,DTI_AD_ext_capsule_L,DTI_AD_cingulum_cg_R,DTI_AD_cingulum_cg_L,DTI_AD_cingulum_hipp_R,DTI_AD_cingulum_hipp_L,DTI_AD_fornix_striaterm_R,DTI_AD_fornix_striaterm_L,DTI_AD_sup_long_fascic_R,DTI_AD_sup_long_fascic_L,DTI_AD_sup_frontocc_fascic_R,DTI_AD_sup_frontocc_fascic_L,DTI_AD_unc_fascic_R,DTI_AD_unc_fascic_L,DTI_AD_tapetum_R,DTI_AD_tapetum_L,DTI_RD_mid_cerebll_pedunc,DTI_RD_pontine_xing_tract,DTI_RD_genu_cc,DTI_RD_body_cc,DTI_RD_splenium_cc,DTI_RD_fornix,DTI_RD_corticospinal_R,DTI_RD_corticospinal_L,DTI_RD_med_lemniscus_R,DTI_RD_med_lemniscus_L,DTI_RD_inf_cerebll_pedunc_R,DTI_RD_inf_cerebll_pedunc_L,DTI_RD_sup_cerebll_pedunc_R,DTI_RD_sup_cerebll_pedunc_L,DTI_RD_cerebr_pedunc_R,DTI_RD_cerebr_pedunc_L,DTI_RD_ant_int_capsule_R,DTI_RD

In [8]:
columns_to_keep = ['Status_x', 'Group', 'TBIID', 'DOB', 'GType', 'Race', 'Hispanic', 'Handedness', 'ScreenAge', 'Education', 'Marital', 'servconn',
                   'cestotal', 'ExpPB', 'ExpMark1', 'ExpAntiM', 'ExpStim',
       'ExpOthrS', 'ExpDEET', 'ExpTick', 'ExpPCollr', 'ExpPStrp',
       'ExpPEnvi', 'ExpToxic', 'ExpPaint', 'ExpXsVib', 'ExpHStrk',
       'ExpRadar', 'ExpIonRa', 'ExpYCake', 'ExpVhicl', 'ExpUrRnd',
       'ExpDtOrd', 'ExpGasM', 'ExpMOPP', 'ExpRadBg', 'ExpAN', 'ExpNG',
       'ExpTNT', 'ExpPETN', 'ExpRDX', 'ExpNC', 'ExpANFO', 'ExpCompB',
       'ExpOctol', 'ExpPntlt', 'ExpDynmt', 'ExpOthrX', 
                   'QKOIorA', 'QKOExpMil', 'QKOAllMil',
       'QKOLife', 'QBlstExp', 'QBEACRM', 'Q5plus2',
                   'MnthSncBlst', 'QEDist1', 'QEDist2', 'QEDist3', 'QEDist4', 'QEDist5',
                   'auditc', 'AUDIT1', 'AUDIT2', 'AUDIT3', 'DOPA', 'DA', 'DOPAC', 'NE', 'DHPG', 'da_dopa_ratio',
       'dopac_da_ratio', 'ne_dopa_ratio', 'dhpg_ne_ratio',
                   'HEIGHT', 'HEIGDEC', 'WEIGHT', 'BMI', 'BPSYS', 'BPDIAS', 'HRATE',
                   'HYPERTEN', 'HYPERCHO', 'DIABETES', 'B12DEF', 'THYROID', 'PITHistTBIID', 'BGlucose', 'BNa', 'BUN', 'BCreat', 'BOsmo', 'UOsmo', 'USG', 'UNa',
       'BK', 'TotalChol', 'LDL', 'HDL', 'Trig', 'CSFPROTEIN_x', 'CSFGLUCOSE', 'CSFRBCS',
       'MHxPain', 'MHxHA', 'MHxHtn', 'MHxCard', 'MHxGI', 'MHxNeuro', 'MHxLung', 'MHxApnea',
                   'SCPTSD', 'SCMDD', 'SCPD', 'SCGAD', 'SCNone',
                   'HYPERTEN', 'HYPERCHO', 'DIABETES', 'B12DEF', 'THYROID', 'DvpVers',
       'DvpHA', 'DvpHAAct', 'DvpHASlp', 'DvpHAMd', 'DvpHAStr', 'DvpBP',
       'DvpBPAct', 'DvpBPslp', 'DvpBPMd', 'DvpBPStr', 
                   'Inhibit_Mean', 'Shift_Mean', 'Emotional Control_Mean',
       'Self Monitor_Mean', 'Initiate_Mean', 'Working Memory_Mean', 'Plan/Organize_Mean', 
        'BISAtt', 'BISMtr', 'BISNonpl', 'BISTot',
                  'capsCrtA', 'b1f', 'b1i', 'b2f',
       'b2i', 'b3f', 'b3i', 'b4f', 'b4i', 'b5f', 'b5i', 'c6f', 'c6i',
       'c7f', 'c7i', 'c8f', 'c8i', 'c9f', 'c9i', 'c10f', 'c10i', 'c11f', 
                  'c12f', 'c12i', 'd13f', 'd13i', 'd14f', 'd14i', 'd15f',
       'd15i', 'd16f', 'd16i', 'd17f', 'd17i', 'f20', 'f21', 'f22',
       'caps23', 'caps24', 'caps25', 'caps26f', 'caps26i', 'caps27f',
       'caps27i', 'caps28f', 'caps28i', 'caps29f', 'caps29i', 'caps30f',
       'caps30i', 'CAPSTotal', 'hrslp', 'PSQI1hr', 'PSQI1min', 'PSQI2',
       'PSQI3hr', 'PSQI3min', 'PSQI5a', 'PSQI5b', 'PSQI5c', 'PSQI5d',
       'PSQI5e', 'PSQI5f', 'PSQI5g', 'PSQI5h', 'PSQI5i', 'PSQI5j',
       'PSQI5jco', 'PSQI6', 'PSQI7', 'PSQI8', 'PSQI9', 'PSQIc1', 'PSQIc2',
       'PSQIc3', 'PSQIc4', 'PSQIc5', 'PSQIc6', 'PSQIc7', 'PSQItot',
       'PCL1', 'PCL2', 'PCL3', 'PCL4', 'PCL5', 'PCL6', 'PCL7', 'PCL8',
       'PCL9', 'PCL10', 'PCL11', 'PCL12', 'PCL13', 'PCL14', 'PCL15',
       'PCL16', 'PCL17', 'PCLTot', 'PHQ1', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ5',
       'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9', 'PHQTot', 
                  'tbiDizzy', 'tbiBalan', 'tbiCoord', 'tbiHeada', 'tbiNaus', 'tbiVision',
       'tbiLight', 'tbiHear', 'tbiNoise', 'tbiTingl', 'tbiTstsml',
       'tbiAppet', 'tbiConc', 'tbiForget', 'tbiDecis', 'tbiSlow',
       'tbiEnergy', 'tbiSleep', 'tbiAnx', 'tbiSad', 'tbiIrrit',
       'tbiOverw', 'tbiDisin', 'tbiWithd', 'tbiRing', 'tbiMoods',
       'tbiFight', 'tbiSpch', 'NSITot', 'TBITot',
                  'PreSleep', 'PreCaff',
       'PreETOH', 'PreNic', 'PreTHC', 'FrmMedications', 'FrmEducation',
       'FrmPretesting', 'FrmMMSE', 'FrmDigitSpan', 'FrmAnimals',
       'FrmTrail', 'FrmFAS', 'FrmTOMM', 'FrmCVMT', 'FrmCVLT-II',
       'FrmTOMMR', 'FrmFinger', 'FrmLNS', 'FrmPegboard', 'FrmCVMTDelay',
       'FrmCVLTRecall', 'FrmACT', 'FrmRuff 2&7', 'FrmCVLT-II_FC',
       'FrmWTAR', 'FrmNDRT', 'FrmgradCPT', 'FrmBVMT-R', 'FrmTEA',
       'FrmDKEFS', 'FrmBVMTRecall', 'FrmBVMTTrial', 'FrmSDMT', 'FrmMIST',
       'FrmWCST', 'FrmFrSBe', 'FrmQOLIBRI', 'FrmDRRI-2', 'FrmMSNQ',
       'FrmFTND', 'FrmPRMQ', 'FrmIPAQ', 'FrmBPAQ', 'FrmBRIEF-A', 'FrmFIS',
       'FrmSF-36v', 'FrmWHOQOL', 'FrmISI', 'FrmPSAS', 'FrmIESS',
       'FrmBSERQ']

first_final = first_visit_data[columns_to_keep]
                   
#create new column for sum of distance from blast (worst 5)
first_final['QEDist_sum'] = first_final.loc[:, 'QEDist1':'QEDist5'].sum(axis=1)
#create new column for mean of distance from blast (worst 5)
first_final['QEDist_mean'] = first_final.loc[:, 'QEDist1':'QEDist5'].mean(axis=1)
#create new column for min of distance from blast (worst 5)
first_final['QEDist_min'] = first_final.loc[:, 'QEDist1':'QEDist5'].min(axis=1)

#create new columns for NSI 4-factor scoring approach
NSI_comp_vestibular = ['tbiDizzy', 'tbiBalan', 'tbiCoord']
NSI_comp_somatosensory = ['tbiHeada', 'tbiNaus', 'tbiVision', 'tbiLight', 'tbiNoise', 'tbiTingl', 'tbiTstsml']
NSI_comp_cognitive = ['tbiConc', 'tbiForget', 'tbiDecis', 'tbiSlow']
NSI_comp_affective = ['tbiEnergy', 'tbiSleep', 'tbiAnx', 'tbiSad', 'tbiIrrit', 'tbiOverw']
NSI_comp_ERP_affective = ['tbiDisin', 'tbiWithd', 'tbiMoods', 'tbiFight']
NIS_comp_ERP_vestsom = ['tbiRing', 'tbiSpch']

first_final['NSI_comp_vestibular'] = first_final.loc[:, NSI_comp_vestibular].sum(axis=1)
first_final['NSI_comp_somatosensory'] = first_final.loc[:, NSI_comp_somatosensory].sum(axis=1)
first_final['NSI_comp_cognitive'] = first_final.loc[:, NSI_comp_cognitive].sum(axis=1)
first_final['NSI_comp_affective'] = first_final.loc[:, NSI_comp_affective].sum(axis=1)
first_final['NSI_comp_ERP_affective'] = first_final.loc[:, NSI_comp_ERP_affective].sum(axis=1)
first_final['NIS_comp_ERP_vestsom'] = first_final.loc[:, NIS_comp_ERP_vestsom].sum(axis=1)
                   
first_final.shape

(144, 342)

In [9]:
#determined outliers for auditc and QBlstExp (outlier = >3 SD from mean) and remove
first_final = first_final[first_final["TBIID"] != 'C010']
first_final = first_final[first_final["TBIID"] != 'T080']

In [10]:
first_final.to_csv('first_final.csv')

In [ ]:
#prepare AUDIT data
AUDITC_cols = ['auditc', 'AUDIT1', 'AUDIT2', 'AUDIT3']

In [ ]:
#get AUDIT counts and percents for chi squared analysis and viz
AUDIT_tot_counts = first_final.groupby('Group')['auditc'].value_counts()
AUDIT_1_counts = first_final.groupby('Group')['AUDIT1'].value_counts()
AUDIT_2_counts = first_final.groupby('Group')['AUDIT2'].value_counts()
AUDIT_3_counts = first_final.groupby('Group')['AUDIT3'].value_counts()
AUDIT_tot_counts.to_csv('AUDIT_tot_counts.csv')
AUDIT_1_counts.to_csv('AUDIT_1_counts.csv')
AUDIT_2_counts.to_csv('AUDIT_2_counts.csv')
AUDIT_3_counts.to_csv('AUDIT_3_counts.csv')

AUDIT_tot_perc = first_final.groupby('Group')['auditc'].value_counts() / first_final.groupby('Group')['auditc'].count()
AUDIT_1_perc = first_final.groupby('Group')['AUDIT1'].value_counts() / first_final.groupby('Group')['AUDIT1'].count()
AUDIT_2_perc = first_final.groupby('Group')['AUDIT2'].value_counts() / first_final.groupby('Group')['AUDIT2'].count()
AUDIT_3_perc = first_final.groupby('Group')['AUDIT3'].value_counts() / first_final.groupby('Group')['AUDIT3'].count()
AUDIT_tot_perc.to_csv('AUDIT_tot_perc.csv')
AUDIT_1_perc.to_csv('AUDIT_1_perc.csv')
AUDIT_2_perc.to_csv('AUDIT_2_perc.csv')
AUDIT_3_perc.to_csv('AUDIT_3_perc.csv')